In [59]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
#!/usr/bin/env python
# coding: utf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

import src.features.pabellon as pb

# Configuración de gráficos
sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

ANIO_INICIO = 2017
ANIO_TERMINO = 2035
COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

In [73]:
# Cargar datos
casos_area_de_influencia = pb.cargar_casos_area_de_influencia(
    "../data/interim/casos_teoricos_diagnosticos.xlsx"
)

# Lee porcentajes y especialidades quirurgicas de trazadoras
porcentajes_de_quirurgicos, especialidades_quirurgicas = pb.cargar_porcentajes_de_quirurgicos(
    "../data/external/incidencias_y_prevalencias_INT.xlsx"
)

# Calcular casos quirúrgicos
DIAGS_AREA_DE_INFLUENCIA = casos_area_de_influencia.index
casos_quirurgicos = pb.calcular_casos_quirurgicos(
    casos_area_de_influencia, porcentajes_de_quirurgicos
)

# Calcula los casos quirurgicos por especialidad para las complicaciones
casos_especialidad = pb.calcular_casos_quirurgicos_por_especialidad(
    casos_quirurgicos, especialidades_quirurgicas
)

Casos del área de influencia cargados y formateados:
+---+-------------+------------------------------------+-------------+----------------------+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|   | Diagnostico |      Diagnosticos Contenidos       | Estadística | Casos (Cada 100.000) |   Edad Inci

In [62]:
# Lee la base de GRD unida con Pabellon
grd_y_pabellon = pd.read_csv("../data/raw/5_duracion_int_q/grd_interno_y_pabellon.csv")
grd_y_pabellon["duracion"] = pd.to_timedelta(grd_y_pabellon["duracion"])

# Extrae los diagnosticos que se van a reasignar
diagnosticos_a_reasignar = casos_area_de_influencia.dropna(subset="Diagnosticos Contenidos")
diagnosticos_a_reasignar = diagnosticos_a_reasignar["Diagnosticos Contenidos"].str.split(", ")
diagnosticos_a_reasignar = diagnosticos_a_reasignar.reset_index()

# Reasigna diagnosticos y los agrupa
grd_y_pabellon = pb.reasignar_diagnosticos(
    grd_y_pabellon, "diag_01_principal_cod", diagnosticos_a_reasignar
)

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['I052', 'I080', 'I081'] a I051
Cambiando ['I352'] a I350
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX


In [63]:
# Obtiene el resumen de duraciones de las int. q por diagnostico acumuladas
resumen_duraciones_int_q_acumuladas = (
    grd_y_pabellon.query("ano_de_egreso >= 2019 ")
    .groupby(["diag_01_principal_cod"])["duracion"]
    .describe()
)

# Obtiene el resumen de duraciones para los diagnosticos relevantes
resumen_duraciones_diags_relevantes = resumen_duraciones_int_q_acumuladas[
    resumen_duraciones_int_q_acumuladas.index.isin(DIAGS_AREA_DE_INFLUENCIA)
]["75%"]

In [64]:
# Calcular tiempo utilizado en pabellón en horas
tiempo_utilizado_pabellon_horas = pb.calcular_tiempo_utilizado_pabellon(
    casos_quirurgicos, resumen_duraciones_diags_relevantes
)

# Calcular horas laborales
horas_laborales = pb.calcular_horas_laborales(ANIO_INICIO, ANIO_TERMINO, 12)

# Calcular cantidad de pabellones necesarios
cantidad_de_pabellones_necesarios = pb.calcular_cantidad_de_pabellones_necesarios(
    tiempo_utilizado_pabellon_horas, horas_laborales
)

# Obtiene la suma total de pabellones
suma_total_pabellones = cantidad_de_pabellones_necesarios.sum()

Tiempo utilizado en pabellón calculado (en horas):
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      |        2017        |        2018        |        2019        |        2020        |        2021        |        2022        |        2023        |        2024        |        2025        |        2026        |        2027        |        2028        |        2029        |        2030        |        2031        |        2032        |        2033        |        2034        |        2035        |
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------

## Complicaciones

In [92]:
# Define las complicaciones a buscar
a_buscar_operaciones = {
    "ecmo": r"ECMO",
    "trasplantes": r"TRASPLANTE|TRANSPLANTE",
    "aseos": r"ASEO",
    "drenajes": r"DRENAJE",
    "traqueostomias": r"TRAQUEOSTOMIA|TRAQUEOSTOMÍA",
    "reintervenciones_reoperaciones": r"REINTERVENCION|REOPERACION|REINTERVENCIÓN|REOPERACIÓN",
}

# Define los diagnosticos que son complicaciones
a_buscar_diagnosticos = {
    "empiemas": r"EMPIEMA"
}

# Busca los nombres de las operaciones en la base de pabellon
resultados_operaciones = pb.iterar_en_complicaciones_a_buscar(
    grd_y_pabellon, a_buscar_operaciones, "intervencion_quirurgica"
)

# Busca los diagnosticos en el primer diagnostico y segundo
resultados_diagnosticos = pb.iterar_en_complicaciones_a_buscar(
    grd_y_pabellon, a_buscar_diagnosticos, "diagnostico"
)

# Obtiene resumen de complicaciones
resumen_complicaciones = pd.concat([resultados_operaciones, resultados_diagnosticos])

In [142]:
# Une los datos de complicaciones y los casos de especialidad
casos_complicaciones = (
    casos_especialidad.merge(resumen_complicaciones, how="left", left_index=True, right_index=True)
    .reset_index()
    .sort_values(["complicacion", "especialidades"])
    .set_index("complicacion")
    .reset_index()
)

# Obtiene los casos de complicaciones
casos_complicaciones[COLUMNAS_POBLACION_INE] = casos_complicaciones[COLUMNAS_POBLACION_INE].mul(
    casos_complicaciones["fraccion"], axis=0
)

# Obtiene el tiempo de pabellon por las complicaciones
tiempo_utilizado_pabellon_complicaciones_horas = casos_complicaciones[COLUMNAS_POBLACION_INE].mul(
    casos_complicaciones["tiempo_operacion_75%"], axis=0
).apply(lambda x: x.dt.total_seconds() / 3600)

In [141]:
tiempo_utilizado_pabellon_complicaciones_horas

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,257.435575,263.274460,269.615577,275.880194,279.519161,281.767328,283.674886,285.464158,287.175539,288.813922,290.376509,291.859124,293.257092,294.567123,295.806940,296.969005,298.047660,299.043389,299.957274
1,277.048807,281.491673,287.131805,293.026819,296.488754,298.560378,300.229653,301.748533,303.168290,304.499624,305.746111,306.912062,307.998358,309.004121,310.147644,311.102273,311.989675,312.817886,313.591666
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,29.177528,29.839302,30.557999,31.268025,31.680463,31.935268,32.151469,32.354264,32.548230,32.733923,32.911025,33.079064,33.237508,33.385986,33.526506,33.658213,33.780467,33.893322,33.996901
4,82.376363,83.697384,85.374394,87.127189,88.156544,88.772511,89.268845,89.720461,90.142605,90.538458,90.909082,91.255760,91.578754,91.877803,92.217813,92.501658,92.765514,93.011770,93.241842
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.712605,0.728768,0.746320,0.763661,0.773734,0.779958,0.785238,0.790191,0.794928,0.799463,0.803789,0.807893,0.811762,0.815389,0.818820,0.822037,0.825023,0.827779,0.830309
7,211.627448,215.021191,219.329481,223.832467,226.476913,228.059350,229.334449,230.494665,231.579165,232.596123,233.548268,234.438896,235.268678,236.036943,236.910439,237.639645,238.317499,238.950139,239.541201
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,234.368494,239.684197,245.457128,251.160415,254.473318,256.520042,258.256676,259.885624,261.443659,262.935238,264.357812,265.707580,266.980285,268.172933,269.301659,270.359598,271.341602,272.248111,273.080108


In [136]:
tiempo_utilizado_pabellon_complicaciones[COLUMNAS_POBLACION_INE].sum()

2017   61 days 12:44:36.607998786
2018   62 days 16:06:37.292996598
2019   64 days 00:32:39.035022361
2020   65 days 09:39:41.506767187
2021   66 days 05:01:26.276640329
2022   66 days 16:45:45.267835613
2023   67 days 02:25:41.527991167
2024   67 days 11:20:18.321937108
2025   67 days 19:45:03.297409439
2026   68 days 03:42:42.625860890
2027   68 days 11:13:37.419356276
2028   68 days 18:18:06.983245809
2029   69 days 00:55:45.593703658
2030   69 days 07:05:57.212731677
2031   69 days 13:34:34.939627965
2032   69 days 19:15:26.261247022
2033   70 days 00:32:05.148064787
2034   70 days 05:26:11.633263123
2035   70 days 09:58:51.149108236
dtype: timedelta64[ns]

In [65]:
# Carga el resumen de area de estudio para MINSAL
resumen_area_de_estudio_minsal = (
    pd.read_excel(
        "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_INT"
    )
    .drop(columns="Porcentaje Hospitalizados de Ambulatorios")
    .sort_values("Diagnostico")
)
resumen_area_de_estudio_minsal["diagnostico_separado"] = (
    resumen_area_de_estudio_minsal["Diagnostico"].str.split(" - ").str[0]
)
resumen_area_de_estudio_minsal = resumen_area_de_estudio_minsal.set_index("diagnostico_separado")

# Sintetiza informacion de pabellon para el 2035
resumen_quirurgicos = pd.DataFrame(
    {
        "porcentaje_quirurgico": porcentajes_de_quirurgicos,
        "casos_quirurgicos_2035": casos_quirurgicos["2035"],
        "tiempo_quirurgico_75%_2019_a_2022": resumen_duraciones_diags_relevantes,
        "horas_pabellon_2035": tiempo_utilizado_pabellon_horas["2035"],
        "horas_laborales_2035_pabellon_12_hrs": horas_laborales["2035"],
        "cantidad_de_pabellones": cantidad_de_pabellones_necesarios["2035"],
    }
)

# Une ambos resumenes segun el diagnostico
resumen_minsal = resumen_area_de_estudio_minsal.merge(
    resumen_quirurgicos, how="left", left_index=True, right_index=True
)

In [66]:
print(f"> Pabellones 2035: {resumen_minsal['cantidad_de_pabellones'].sum():.2f}")

> Pabellones 2035: 5.44


In [67]:
a_guardar = {
    "casos_area_de_influencia": casos_area_de_influencia,
    "porcentajes_quirurgicos": porcentajes_de_quirurgicos,
    "casos_quirurgicos": casos_quirurgicos,
    "resumen_duraciones_int_q_rel": resumen_duraciones_diags_relevantes,
    "tiempo_utilizado_pabellon": tiempo_utilizado_pabellon_horas,
    "horas_laborales_por_anio": horas_laborales,
    "cantidad_de_pabellones_desg": cantidad_de_pabellones_necesarios,
    "suma_total_pabellones": suma_total_pabellones,
    "resumen_MINSAL": resumen_minsal,
}

In [68]:
with pd.ExcelWriter("../data/interim/estimacion_pabellones_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)